# Edge Ordering

Edge ordering makes a difference on the on the join tree and potentials produced. Let's take the BBN network structure below where all nodes are binary having the values `on` and `off`.

a --> c <-- b

Note how `c` has 2 parents, `a` and `b`. The potential (or conditional probability table `CPT`) for `c` is specified as a list of probabilities as follows.

```python
[0.7, 0.3, 0.2, 0.8, 0.6, 0.4, 0.6, 0.4]
```

Let's say that this list of probabilities represents the CPT below.

```
|       |       | c=on | c=off |
|-------|-------|------|-------|
| a=on  | b=on  | 0.7  | 0.3   |
| a=on  | b=off | 0.2  | 0.8   |
| a=off | b=on  | 0.6  | 0.4   |
| a=off | b=off | 0.6  | 0.4   |
```

When we define a BBN structure (be it programmatically in code/Python or declaratively in JSON), we should define and add the edge `a -> c` to the graph before the edge `b -> c`. Below is the code where we do so.

In [1]:
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable

def get_bbn1():
    a = BbnNode(Variable(0, 'a', ['on', 'off']), [0.2, 0.8])
    b = BbnNode(Variable(1, 'b', ['on', 'off']), [0.8, 0.2])
    c = BbnNode(Variable(2, 'c', ['on', 'off']), [0.7, 0.3, 0.2, 0.8, 0.6, 0.4, 0.6, 0.4])

    bbn = Bbn() \
        .add_node(a) \
        .add_node(b) \
        .add_node(c) \
        .add_edge(Edge(a, c, EdgeType.DIRECTED)) \
        .add_edge(Edge(b, c, EdgeType.DIRECTED))

    return bbn

When we add the edge `b -> c` to the network structure before `a -> c`, then the induced CPT for `c` will be as follows. This second CPT for `c` is not the same at all for the first one!

```
|       |       | c=on | c=off |
|-------|-------|------|-------|
| b=on  | a=on  | 0.7  | 0.3   |
| b=on  | a=off | 0.2  | 0.8   |
| b=off | a=on  | 0.6  | 0.4   |
| b=off | a=off | 0.6  | 0.4   |
```

Here is the code for creating a BBN where we add `b -> c` before `a -> c`.

In [2]:
def get_bbn2():
    a = BbnNode(Variable(0, 'a', ['on', 'off']), [0.2, 0.8])
    b = BbnNode(Variable(1, 'b', ['on', 'off']), [0.8, 0.2])
    c = BbnNode(Variable(2, 'c', ['on', 'off']), [0.7, 0.3, 0.2, 0.8, 0.6, 0.4, 0.6, 0.4])

    bbn = Bbn() \
        .add_node(a) \
        .add_node(b) \
        .add_node(c) \
        .add_edge(Edge(b, c, EdgeType.DIRECTED)) \
        .add_edge(Edge(a, c, EdgeType.DIRECTED))

    return bbn

Although the networks (regardless of the order of how we add the edges) are the same in both cases, the parameters induced are NOT and sensitive to the order of how the edges are added. Now, let's compare the posteriors of of these 2 BBNs.

In [5]:
from pybbn.pptc.inferencecontroller import InferenceController

b1 = get_bbn1()
b2 = get_bbn2()

j1 = InferenceController.apply(b1)
j2 = InferenceController.apply(b2)

Here are the posteriors for the first BBN. Note that the id-to-name as defined above are as follows.

- 0: a
- 1: b
- 2: c

Keep an eye on id 2, thus.

In [6]:
for node in j1.get_bbn_nodes():
    potential = j1.get_bbn_potential(node)
    print(potential)
    print('-' * 10)

1=on|0.80000
1=off|0.20000
----------
2=on|0.60000
2=off|0.40000
----------
0=on|0.20000
0=off|0.80000
----------


Here are the posteriors for the second BBN.

In [7]:
for node in j2.get_bbn_nodes():
    potential = j2.get_bbn_potential(node)
    print(potential)
    print('-' * 10)

1=on|0.80000
1=off|0.20000
----------
2=on|0.36000
2=off|0.64000
----------
0=on|0.20000
0=off|0.80000
----------


For now, there is no workaround for this issue of logically identical specified BBNs producing different potentials as a result of edge insertion order. Just make sure you are aware and careful.